In [1]:
from pathlib import Path

root_results_path = "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences"
# Define the directory to search
directory = Path(root_results_path)

# Use rglob to search recursively ('**/*.csv') 
# or glob for just the current folder ('*.csv')
csv_files = []
for path in list(directory.rglob('*.csv')):
    if "tests" in str(path) or len(str(path).split('/')[-1])<20:
        continue
    csv_files.append(path)
csv_files

[PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_DSC_DAsafe_epoch_80/no_TTA/overlap_0.5/th_0.4/BCE_DSC_DAsafe_epoch_80_no_TTA_overlap_0.5_th_0.4.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_DSC_DAsafe_epoch_80/no_TTA/overlap_0.5/th_0.5/BCE_DSC_DAsafe_epoch_80_no_TTA_overlap_0.5_th_0.5.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/BCE_DSC_DAsafe_epoch_80/no_TTA/overlap_0.5/th_0.6/BCE_DSC_DAsafe_epoch_80_no_TTA_overlap_0.5_th_0.6.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_CLDICE_DAsafe_epoch_40/TTA/overlap_0.5/th_0.4/Focal_Tversky_CLDICE_DAsafe_epoch_40_TTA_overlap_0.5_th_0.4.csv'),
 PosixPath('/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_CLDICE_DAsafe_epoch_40/TTA/overlap_0.5/th_0.7/Focal_Tversky_CLDICE_DAsafe_epoch_40_TTA_overlap_0.5_th_0.7.csv'),
 Posi

In [2]:
import pandas as pd
import glob
import os
from datetime import datetime

rows_list = []
cols_to_remove = ['tif_paths', 'id', 'pred_paths']

for file in csv_files:
    filename = os.path.basename(file)
    c_time = os.path.getctime(file)
    dt_c_time = datetime.fromtimestamp(c_time)
    df = pd.read_csv(file)
    
    if len(df) >= 80:
        mean_row = df.iloc[[79]].copy()
        clean_row = mean_row.drop(columns=cols_to_remove, errors='ignore')
        
        # --- PARSING FILENAME ---
        clean_row['Losses'] = filename.split('_DA')[0] if '_DA' in filename else  filename.split('_epoch')[0]
        clean_row['epoch'] = filename.split('epoch_')[-1].split('_')[0] 
        clean_row['Train DA'] = filename.split('_DA')[-1].split('_')[0] if '_DA' in filename else  filename.split('_epoch')[0][-4:]
        clean_row['tta_status'] = 'no_TTA' if 'no_TTA' in filename else 'TTA'
        
        if 'overlap_' in filename:
            overlap_val = filename.split('overlap_')[1].split('_')[0]
            clean_row['overlap'] = float(overlap_val)
            
        if 'th_' in filename:
            th_val = filename.split('th_')[1].replace('.csv', '')
            clean_row['threshold'] = float(th_val)
        clean_row['dt_c_time'] = dt_c_time
        rows_list.append(clean_row)

# 2. Create the new combined DataFrame
combined_df = pd.concat(rows_list, ignore_index=True)

# 3. REORDER COLUMNS: Define the leading order
# We list exactly what we want first, then add the rest of the columns
fixed_cols = ['Losses', 'epoch', 'Train DA', 'tta_status', 'overlap', 'threshold']
other_cols = [c for c in combined_df.columns if c not in fixed_cols]

combined_df = combined_df[fixed_cols + other_cols]

# 4. View the result
combined_df.sort_values(['image_score'], ascending=False)
#combined_df.sort_values(['Losses', 'epoch', 'tta_status', 'image_score'], ascending=False)

,Losses,epoch,Train DA,tta_status,overlap,threshold,image_score,topo_score,surface_dice,voi_score,voi_split,voi_merge,dt_c_time
53,BCE_Tversky,model,safe,TTA,0.5,0.40,0.538749,0.174875,0.829974,0.559414,1.194520,1.461276,2026-02-06 05:09:51.368912
45,Focal_Tversky,150,safe,TTA,0.5,0.40,0.537427,0.186253,0.812687,0.563173,1.178807,1.439129,2026-01-29 22:07:55.728090
54,BCE_Tversky,model,safe,TTA,0.5,0.50,0.535272,0.170764,0.832006,0.550974,1.313335,1.439372,2026-02-06 09:16:18.336620
47,Focal_Tversky,150,safe,TTA,0.5,0.50,0.531597,0.176505,0.817828,0.549730,1.333118,1.444002,2026-01-30 05:32:26.873576
15,Focal_Tversky,model,safe,TTA,0.6,0.40,0.529340,0.168734,0.804973,0.562799,1.189067,1.431590,2026-02-01 12:13:00.509741
16,Focal_Tversky,model,safe,TTA,0.5,0.40,0.528567,0.166644,0.804339,0.563013,1.170087,1.447955,2026-01-31 18:58:03.464306
55,BCE_Tversky,model,safe,no_TTA,0.5,0.40,0.528368,0.147174,0.825670,0.557805,1.147804,1.523732,2026-02-05 19:48:45.475322
0,BCE_DSC,80,safe,no_TTA,0.5,0.40,0.526443,0.143722,0.832690,0.548242,1.317439,1.477125,2026-02-04 15:14:45.340866
56,BCE_Tversky,model,safe,no_TTA,0.5,0.50,0.525515,0.140162,0.829888,0.551445,1.238467,1.506798,2026-02-05 23:28:51.367756
17,Focal_Tversky,model,safe,TTA,0.5,0.50,0.524729,0.157820,0.815751,0.548201,1.357447,1.429950,2026-02-01 00:49:34.597212


# check individual results

In [3]:
df_1 = pd.read_csv("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_CLDICE_DAsafe_epoch_40/no_TTA/overlap_0.5/th_0.5/Focal_Tversky_CLDICE_DAsafe_epoch_40_no_TTA_overlap_0.5_th_0.5.csv")
df_1.sort_values("topo_score", ascending=False)

,tif_paths,id,pred_paths,image_score,topo_score,surface_dice,voi_score,voi_split,voi_merge
19,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,2961547523,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.609153,0.345385,0.885777,0.558617,1.185232,1.448550
76,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,2821927657,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.587089,0.298709,0.866888,0.554471,1.516604,1.161796
15,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3694884789,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.590520,0.277785,0.855989,0.593110,0.673306,1.613456
55,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3922476405,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.572199,0.245832,0.882407,0.541736,1.476135,1.343594
30,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1845578058,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.593638,0.241246,0.911542,0.577784,1.161015,1.274822
...,...,...,...,...,...,...,...,...,...
49,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1796762532,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.388094,0.014599,0.570423,0.525903,1.277700,1.727268
11,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1734818300,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.400741,0.009866,0.568772,0.567745,0.302300,2.235549
39,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,1522833038,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.433046,0.009090,0.564910,0.664573,0.395800,1.286618
35,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,3681179135,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.464778,0.004271,0.657626,0.666650,0.486012,1.180783


In [4]:
df_2 = pd.read_csv(
    "/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences/Focal_Tversky_DAsafe_epoch_150/TTA/overlap_0.5/th_0.5/Focal_Tversky_DAsafe_epoch_150_TTA_overlap_0.5_th_0.5.csv")
df_2[df['id'] == "2961547523"]

,tif_paths,id,pred_paths,image_score,topo_score,surface_dice,voi_score,voi_split,voi_merge
19,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,2961547523,/home/shadowtwin/Desktop/AI_work/Vesuvius_Chal...,0.607875,0.289979,0.929437,0.558797,1.268537,1.363322


In [5]:
# Merge the two dataframes on the 'id' column
combined_df = df_1.merge(df_2[['id', 'voi_score']], on='id', suffixes=('_1', '_2'))

combined_df['diff'] = combined_df['voi_score_1'] - combined_df['voi_score_2']
combined_df = combined_df.sort_values('diff', ascending=False)
combined_df = combined_df[['id', 'voi_score_1', 'voi_score_2', 'diff']]
print(f"mean: {combined_df['diff'].mean()}")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(combined_df)



mean: 0.019040323141840013


,id,voi_score_1,voi_score_2,diff
54,1471460767,0.683634,0.508196,0.175439
11,1734818300,0.567745,0.431005,0.136739
22,4105398542,0.593891,0.461752,0.132139
43,3198627305,0.619629,0.491474,0.128155
21,3608009641,0.600676,0.476219,0.124458
12,803213133,0.571838,0.453523,0.118315
61,2708764018,0.635725,0.539054,0.096672
69,2203617984,0.608410,0.512796,0.095614
36,1332121747,0.569555,0.483264,0.086292
50,2116132949,0.639729,0.561333,0.078396


# Copy all csv to an independent folder

In [ ]:
from pathlib import Path
import shutil

# Define directories
base_path = Path("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/inferences")
dest_root = Path("/home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results")

# Find all .csv files
csv_files = list(base_path.rglob("*.csv"))

for file in csv_files:
    file_str = str(file)
    
    # Filtering logic
    if '/tests/' in file_str or file.name.startswith('th_'):
        continue
    
    # Create the new path using pathlib (cleaner than .replace)
    relative_path = file.relative_to(base_path)
    dest_path = dest_root / relative_path

    # Create the folder structure in the destination root
    dest_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Copy the file
    shutil.copy2(file, dest_path)
    print(f"Copied: {dest_path}")

Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/BCE_DSC_DAsafe_epoch_80/no_TTA/overlap_0.5/th_0.4/BCE_DSC_DAsafe_epoch_80_no_TTA_overlap_0.5_th_0.4.csv
Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/BCE_DSC_DAsafe_epoch_80/no_TTA/overlap_0.5/th_0.5/BCE_DSC_DAsafe_epoch_80_no_TTA_overlap_0.5_th_0.5.csv
Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/BCE_DSC_DAsafe_epoch_80/no_TTA/overlap_0.5/th_0.6/BCE_DSC_DAsafe_epoch_80_no_TTA_overlap_0.5_th_0.6.csv
Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/Focal_Tversky_CLDICE_DAsafe_epoch_40/TTA/overlap_0.5/th_0.4/Focal_Tversky_CLDICE_DAsafe_epoch_40_TTA_overlap_0.5_th_0.4.csv
Copied: /home/shadowtwin/Desktop/AI_work/Vesuvius_Challenge/Vesuvius/csv_results/Focal_Tversky_CLDICE_DAsafe_epoch_40/TTA/overlap_0.5/th_0.7/Focal_Tversky_CLDICE_DAsafe_epoch_40_TTA_overlap_0.5_th_0.7.csv
Copied: /home/shadowtwin/Desktop/AI